In [15]:
from typing import Dict, TypedDict
from langgraph.graph import StateGraph, START, END

In [16]:
class AgentState(TypedDict):
    num1 : int
    num2 : int
    num3 : int
    d : int
    operation : str
    result : int

In [17]:
def addition1(state:AgentState):
    state['num2'] = state['num1'] + state['num2']
    state['d'] = 2
    return state

def subtraction1(state:AgentState):
    state['num2'] = state['num1'] - state['num2']
    state['d'] = 2
    return state

def addition2(state:AgentState):
    state['result'] = state['num2'] + state['num3']
    return state

def subtraction2(state:AgentState):
    state['result'] = state['num2'] - state['num3']
    return state

def decide_next_move(state:AgentState):
    if state['operation'] == '+' and state['d'] == 1:
        return "add_operation_1"
    elif state['operation'] == '+' and state['d'] == 2:
        return "add_operation_2"
    
    elif state['operation'] == '-' and state['d'] == 1:
        return "sub_operation_1"
    elif state['operation'] == '-' and state['d'] == 2:
        return "sub_operation_2"



In [23]:
graph = StateGraph(AgentState)

graph.add_node("addition1", addition1)
graph.add_node("addition2", addition2)
graph.add_node("subtraction1", subtraction1)
graph.add_node("subtraction2", subtraction2)
graph.add_node("router", lambda state:state)

graph.add_edge(START, "router")

graph.add_conditional_edges(
    "router",
    decide_next_move,
    {
        "add_operation_1" : "addition1",
        "add_operation_2" : "addition2",
        "sub_operation_1" : "subtraction1",
        "sub_operation_2" : "subtraction2"
    }
)

graph.add_edge("addition1", "router")
graph.add_edge("subtraction1", "router")
graph.add_edge("addition2", END)
graph.add_edge("subtraction2", END)

app = graph.compile()

In [20]:
result = app.invoke({"num1" : 12, "num2" : 5, "num3" : 3, "d" : 1, "operation" : '+'})

In [21]:
result

{'num1': 12, 'num2': 17, 'num3': 3, 'd': 2, 'operation': '+', 'result': 20}